In [3]:
import time
import os
import pandas as pd
import numpy as np
import joblib
import mlflow
import dagshub
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.base import clone

dagshub.init(repo_owner='abdsalam25', repo_name='churn-project', mlflow=True)
mlflow.set_experiment("Optuna_Churn_Optimization")

try:
    df = pd.read_csv("../churn_data.csv")
except FileNotFoundError:
    df = pd.read_csv("churn_data.csv")

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
df = df.dropna()

X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn']

categorical_cols = X.select_dtypes(include=['object', 'bool']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessing = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

def objective_ridge(trial, preprocessing, X_train, y_train, use_pca):
    alpha = trial.suggest_float("alpha", 0.1, 10.0, log=True)
    steps = [('prep', clone(preprocessing))]
    if use_pca:
        n_components = trial.suggest_float("n_components", 0.8, 0.99)
        steps.append(('pca', PCA(n_components=n_components)))
    steps.append(('clf', RidgeClassifier(alpha=alpha)))
    pipeline = Pipeline(steps)
    return cross_val_score(pipeline, X_train, y_train, cv=3, scoring="f1").mean()

def objective_gb(trial, preprocessing, X_train, y_train, use_pca):
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    max_depth = trial.suggest_int("max_depth", 3, 8)
    steps = [('prep', clone(preprocessing))]
    if use_pca:
        n_components = trial.suggest_float("n_components", 0.8, 0.99)
        steps.append(('pca', PCA(n_components=n_components)))
    steps.append(('clf', GradientBoostingClassifier(
        learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, random_state=42)))
    pipeline = Pipeline(steps)
    return cross_val_score(pipeline, X_train, y_train, cv=3, scoring="f1").mean()

def objective_xgb(trial, preprocessing, X_train, y_train, use_pca):
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    max_depth = trial.suggest_int("max_depth", 3, 8)
    steps = [('prep', clone(preprocessing))]
    if use_pca:
        n_components = trial.suggest_float("n_components", 0.8, 0.99)
        steps.append(('pca', PCA(n_components=n_components)))
    steps.append(('clf', XGBClassifier(
        use_label_encoder=False, eval_metric='logloss', learning_rate=learning_rate,
        n_estimators=n_estimators, max_depth=max_depth, random_state=42)))
    pipeline = Pipeline(steps)
    return cross_val_score(pipeline, X_train, y_train, cv=3, scoring="f1").mean()

def objective_lgbm(trial, preprocessing, X_train, y_train, use_pca):
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    num_leaves = trial.suggest_int("num_leaves", 20, 50)
    steps = [('prep', clone(preprocessing))]
    if use_pca:
        n_components = trial.suggest_float("n_components", 0.8, 0.99)
        steps.append(('pca', PCA(n_components=n_components)))
    steps.append(('clf', LGBMClassifier(
        learning_rate=learning_rate, n_estimators=n_estimators, num_leaves=num_leaves,
        random_state=42, verbose=-1)))
    pipeline = Pipeline(steps)
    return cross_val_score(pipeline, X_train, y_train, cv=3, scoring="f1").mean()

objectives = {
    "Ridge": objective_ridge,
    "GradientBoosting": objective_gb,
    "XGBoost": objective_xgb,
    "LightGBM": objective_lgbm
}

best_global_f1 = 0
best_global_pipeline = None
best_global_name = ""

for model_name, obj_func in objectives.items():
    for use_pca in [False, True]:
        run_name = f"{model_name}_Optuna_PCA_{use_pca}"
        print(f"Optimizing {run_name}...")
        
        study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
        study.optimize(lambda trial: obj_func(trial, preprocessing, X_train, y_train, use_pca), n_trials=10)
        
        best_params = study.best_params
        steps = [('prep', clone(preprocessing))]
        
        if use_pca:
            steps.append(('pca', PCA(n_components=best_params["n_components"])))
            
        if model_name == "Ridge":
            clf = RidgeClassifier(alpha=best_params["alpha"])
        elif model_name == "GradientBoosting":
            clf = GradientBoostingClassifier(learning_rate=best_params["learning_rate"], 
                                           n_estimators=best_params["n_estimators"], 
                                           max_depth=best_params["max_depth"], random_state=42)
        elif model_name == "XGBoost":
            clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss',
                              learning_rate=best_params["learning_rate"],
                              n_estimators=best_params["n_estimators"],
                              max_depth=best_params["max_depth"], random_state=42)
        elif model_name == "LightGBM":
            clf = LGBMClassifier(learning_rate=best_params["learning_rate"],
                               n_estimators=best_params["n_estimators"],
                               num_leaves=best_params["num_leaves"], random_state=42, verbose=-1)
        
        steps.append(('clf', clf))
        final_pipeline = Pipeline(steps)
        final_pipeline.fit(X_train, y_train)
        
        y_pred = final_pipeline.predict(X_test)
        test_f1 = f1_score(y_test, y_pred)
        
        if test_f1 > best_global_f1:
            best_global_f1 = test_f1
            best_global_pipeline = final_pipeline
            best_global_name = run_name

        with mlflow.start_run(run_name=run_name):
            mlflow.log_params(best_params)
            mlflow.log_param("pca", use_pca)
            mlflow.log_metric("cv_f1", study.best_value)
            mlflow.log_metric("test_f1", test_f1)
            mlflow.sklearn.log_model(final_pipeline, "model")

models_dir = "../models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

save_path = os.path.join(models_dir, "best_model_optuna.joblib")
joblib.dump(best_global_pipeline, save_path)

print(f"Global Best Model: {best_global_name}")
print(f"Test F1 Score: {best_global_f1}")
print(f"Model saved to: {save_path}")

Accessing as abdsalam25

Initialized MLflow to track repo "abdsalam25/churn-project"

Repository abdsalam25/churn-project initialized!

2025/12/17 21:02:11 INFO mlflow.tracking.fluent: Experiment with name 'Optuna_Churn_Optimization' does not exist. Creating a new experiment.
[I 2025-12-17 21:02:11,317] A new study created in memory with name: no-name-e834d755-0d0c-40b0-a356-7fecb3b11d41
[I 2025-12-17 21:02:11,379] Trial 0 finished with value: 0.5776023295647147 and parameters: {'alpha': 0.5611516415334507}. Best is trial 0 with value: 0.5776023295647147.
[I 2025-12-17 21:02:11,428] Trial 1 finished with value: 0.5776003433913804 and parameters: {'alpha': 7.969454818643932}. Best is trial 0 with value: 0.5776023295647147.


Optimizing Ridge_Optuna_PCA_False...


...

Optimizing LightGBM_Optuna_PCA_False...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 21:04:47,328] Trial 0 finished with value: 0.5543789604222434 and parameters: {'learning_rate': 0.11861663446573512, 'n_estimators': 193, 'num_leaves': 42}. Best is trial 0 with value: 0.5543789604222434.
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/uti

🏃 View run LightGBM_Optuna_PCA_False at: https://dagshub.com/abdsalam25/churn-project.mlflow/#/experiments/1/runs/b2e3b9e1c5064c1ea661b5d40fda0d7b
🧪 View experiment at: https://dagshub.com/abdsalam25/churn-project.mlflow/#/experiments/1
Optimizing LightGBM_Optuna_PCA_True...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 21:05:06,334] Trial 0 finished with value: 0.5273478966392485 and parameters: {'learning_rate': 0.11861663446573512, 'n_estimators': 193, 'num_leaves': 42, 'n_components': 0.913745111997437}. Best is trial 0 with value: 0.5273478966392485.
/Library/Frameworks/Python.framework/Versions/3.13/lib/p

🏃 View run LightGBM_Optuna_PCA_True at: https://dagshub.com/abdsalam25/churn-project.mlflow/#/experiments/1/runs/943d3f08d73e488e9afa9c6c86a3d123
🧪 View experiment at: https://dagshub.com/abdsalam25/churn-project.mlflow/#/experiments/1
Global Best Model: Ridge_Optuna_PCA_True
Test F1 Score: 0.5802650957290133
Model saved to: ../models/best_model_optuna.joblib


In [2]:
!pip install optuna 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [optuna]
